# Week 4 Lab: RNA-seq

This week's exercises consist of two parts:
* Part 1 (10 pts): review how to quantify gene expression using RNA-sequencing data.
* Part 2 (10 pts): Quiz review for this week's quiz! 

As usual, you may work with a partner or consult your classmates for help.

<font color="red">Reminder, Quiz 2 will be held at the beginning of the second lab section this week</font>.

# Part 1 (10 pts)

## 1. How to measure expression

There are two main metrics used to quantify gene expression using RNA-seq:

### 1. RPKM/FPKM
**RPKM** ("reads per kilobase per million reads"): This is defined as:
$$ RPKM_i = \frac{N_i}{(L_i/1000)(R/1000000)}$$

where $RPKM_i$ is the RPKM value for gene $i$, $N_i$ is the number of reads mapping to gene $i$, $L_i$ is the length of gene $i$ (in base pairs) and $R$ is the total number of reads sequenced.

**FPKM** is a slight modification of RPKM to deal with paired end reads. FPKM gives "fragments per kilobase per million reads", where both reads in a pair belong to the same fragment and so are only counted once.

### 2. TPM
**TPM** ("transcripts per million") is computed by:

1. Computing the reads per kilobase (RPK) for each gene $i$ as $RPK_i = \frac{N_i}{(L_i/1000)}$
2. Determining a "per million" scaling factor defined as $S = \frac{\sum_i RPK_i}{1000000}$
3. Normalizing $RPK$ values by this factor: $TPM_i = \frac{RPK_i}{S}$.

Putting this together, we get:
$$ TPM_i = \frac{N_i}{(L_i/1000)(\sum_i RPK_i/1000000)} $$

The main difference between these metrics is how we scale the length-normalized read counts. For TPM, counts will always sum to 1 million and should be proportional to the percent of transcripts in the sample coming from each gene, making it easier to compare values across experiments. RPKM/FPKM do not have quite as straightforward of an interpretion. 

In practice, you will likely achieve similar results using either of these metrics. For a discussion of why TPM is a better metric, see this enjoyable blog post by Lior Pachter: https://liorpachter.wordpress.com/tag/tpm/ and the discussion below (optional reading) that expands on this post.

## 2. Computing TPM/RPKM

We'll go through some examples computing and interpreting TPKM and RPKM. We'll start by writing functions to compute these values.

Consider a sample with five genes expressed:

* Gene A is length 5kb and has 1,000 copies expressed.
* Gene B is length 3kb and has 200 copies expressed.
* Gene C is length 2kb and has 500 copies expressed.
* Gene D is length 10kb and has 50 copies expressed.
* Gene E is length 7kb and has 1 copy expressed.

We perform RNA-sequencing on this sample and count the number of reads mapping to each gene (given in the list `reads_ctrl` below).

**Question 1 (7 pts)** Complete the functions `RPKM` and `TPM` below and use these functions to compute RPKM and TPM values for each of the five genes. Examples are given in the autograder tests so you can make sure your solution is working.

You should find that the magnitudes of RPKM and TPM are quite different, but the relative difference between genes should be similar (e.g. for both metrics, Gene E has the smallest expression level and Gene A has the largest).

In [4]:
import numpy as np

def RPKM(read_counts, gene_lengths):
    """ Function to compute RPKM for each gene
    
    Parameters
    ----------
    read_counts: list of int
       List of read counts for each gene. e.g. [readcount1, readcount2, ...] 
    gene_lengths: list of int
       List of gene lengths for each gene. e.g. [length1, length2, ...]
    
    Returns
    -------
    rpkms : list of float
       List of rpkms for each gene. e.g. [rpkm1, rpkm2, ...]
    """
    rpkms = [0]*len(read_counts) # return list of rpkm values, one for each gene
    # YOUR CODE HERE
    for i in range(len(read_counts)):
        rpkms[i] = read_counts[i]/((gene_lengths[i]/1000)*(sum(read_counts)/1000000))
    #raise NotImplementedError()
    return rpkms

def TPM(read_counts, gene_lengths):
    """ Function to compute TPM for each gene
    
    Parameters
    ----------
    read_counts: list of int
       List of read counts for each gene. e.g. [readcount1, readcount2, ...] 
    gene_lengths: list of int
       List of gene lengths for each gene. e.g. [length1, length2, ...]
    
    Returns
    -------
    tpms : list of float
       List of tpms for each gene. e.g. [tpm1, tpm2, ...]
    """
    tpms = [0]*len(read_counts)
    # YOUR CODE HERE
    rpk = 0
    for i in range(len(read_counts)):
        rpk += read_counts[i]/(gene_lengths[i]/1000)
    for i in range(len(read_counts)):
        tpms[i] = read_counts[i]/((gene_lengths[i]/1000)*(rpk/1000000))
    return tpms

# Gene info (names, lengths, and transcript counts)
txnames = ["A","B","C","D","E"]
txlens = [5000, 3000, 2000, 10000, 7000]
txcounts_ctrl = [1000, 200, 500, 50, 1]
# Number of reads observed for each gene
reads_ctrl = [txcounts_ctrl[i]*txlens[i] for i in range(len(txlens))]

# Compute rpkms and tpms
rpkms_ctrl = RPKM(reads_ctrl, txlens)
tpms_ctrl = TPM(reads_ctrl, txlens)

print("\nRPKM/TMP for each gene")
for i in range(len(txlens)):
    print("Gene %s: RPKM=%.2f TPM=%.2f"%(txnames[i], rpkms_ctrl[i], tpms_ctrl[i]))


RPKM/TMP for each gene
Gene A: RPKM=140706.35 TPM=571102.23
Gene B: RPKM=28141.27 TPM=114220.45
Gene C: RPKM=70353.17 TPM=285551.11
Gene D: RPKM=7035.32 TPM=28555.11
Gene E: RPKM=140.71 TPM=571.10


In [5]:
"""Check output of TPM/RPKM functions"""
test_txlens = [400, 300, 20, 10000, 70000]
test_txcounts_ctrl = [100, 2000, 800, 2, 1]
test_reads_ctrl = [test_txcounts_ctrl[i]*test_txlens[i] for i in range(len(test_txlens))]
test_tpms = TPM(test_reads_ctrl, test_txlens)
test_rpkms = RPKM(test_reads_ctrl, test_txlens)
tpm_results = [34447, 688942, 275576, 688, 344]
rpkm_results = [134048, 2680965, 1072386, 2680, 1340]
for i in range(len(reads_ctrl)):
    assert(abs(test_tpms[i]-tpm_results[i])<1)
    assert(abs(test_rpkms[i]-rpkm_results[i])<1)
assert(round(sum(test_tpms))==1000000)

Now we treat the sample with a drug, which doubles the expression of Gene E (from 1 to 2 copies). We perform RNA-sequencing on the sample before and after treatment using 1 million  reads in each experiment and perform differential expression analysis to see if we can detect any genes whose expression changed.

**Question 2 (3 pts)**: Complete the function `FoldChange` below which computes the fold change in expression for each gene by comparing expression values in a "treatment" vs. "control" sample. Use this function to determine the fold change in expression of each gene calculated based on either the TPM or RPKM metrics. Examples are given in the autograder tests to check your answer is correct

In [9]:
def FoldChange(expr_ctrl, expr_treat):
    """ Compute fold change for each gene in treatment vs. control
    
    Parameters
    ----------
    expr_ctrl : list of float
        List of expression values for each gene in the control experiment
    expr_treat : list of float
        List of expression values for each gene in the treatment experiment
        
    Returns
    -------
    foldchanges : list of float
        List of fold changes (treat/ctrl) for each gene
    """
    foldchanges = []
    # YOUR CODE HERE
    for i in range(len(expr_ctrl)):
        foldchanges.append(expr_treat[i]/expr_ctrl[i])
    return foldchanges

# Gene info (names, lengths, and transcript counts)
txnames = ["A","B","C","D","E"]
txlens = [5000, 3000, 2000, 10000, 7000]
txcounts_ctrl = [1000, 200, 500, 50, 1]
txcounts_treat = [1000, 200, 500, 50, 2]

# Number of reads observed for each gene
reads_ctrl = [txcounts_ctrl[i]*txlens[i] for i in range(len(txlens))]
reads_ctrl = [reads_ctrl[i]*1000000/np.sum(reads_ctrl) for i in range(len(txlens))]
reads_treat = [txcounts_treat[i]*txlens[i] for i in range(len(txlens))]
reads_treat = [reads_treat[i]*1000000/np.sum(reads_treat) for i in range(len(txlens))]

# Compute rpkms and tpms
rpkms_ctrl = RPKM(reads_ctrl, txlens)
tpms_ctrl = TPM(reads_ctrl, txlens)
rpkms_treat = RPKM(reads_treat, txlens)
tpms_treat = TPM(reads_treat, txlens)

fc_rpkm = FoldChange(rpkms_ctrl, rpkms_treat)
fc_tpm = FoldChange(tpms_ctrl, tpms_treat)

print("\nFold change deteted for each gene")
for i in range(len(txlens)):
    print("Gene %s: FoldChange[RPKM]=%.2f FoldChange[TPM]=%.2f"%(txnames[i], fc_rpkm[i], fc_tpm[i]))          


Fold change deteted for each gene
Gene A: FoldChange[RPKM]=1.00 FoldChange[TPM]=1.00
Gene B: FoldChange[RPKM]=1.00 FoldChange[TPM]=1.00
Gene C: FoldChange[RPKM]=1.00 FoldChange[TPM]=1.00
Gene D: FoldChange[RPKM]=1.00 FoldChange[TPM]=1.00
Gene E: FoldChange[RPKM]=2.00 FoldChange[TPM]=2.00


In [10]:
"""Test output of FoldChange"""
values_ctrl = [1,2,3,4,5]
values_test = [2,1,6,12,50]
fc = FoldChange(values_ctrl, values_test)
assert(fc==[2, 0.5, 2, 3, 10])

You should hopefully see the expected result: there was no change in expression of Genes A, B, C, and D (fold change=1), but Gene E doubled its expression (fold change=2)! 

This worked well, since Gene E's expression level is pretty small compared to the total expression of all genes in our sample. Let's see what happens when we see a huge change in expression. Below, we look at the fold changes we would compute using RPKM or TPM if Gene E actually increadsed its expression 100-fold.

In [11]:
# Gene info (names, lengths, and transcript counts)
txnames = ["A","B","C","D","E"]
txlens = [5000, 3000, 2000, 10000, 7000]
txcounts_ctrl = [1000, 200, 500, 50, 1]
txcounts_treat = [1000, 200, 500, 50, 100]

# Number of reads observed for each gene
reads_ctrl = [txcounts_ctrl[i]*txlens[i] for i in range(len(txlens))]
reads_ctrl = [reads_ctrl[i]*1000000/np.sum(reads_ctrl) for i in range(len(txlens))]
reads_treat = [txcounts_treat[i]*txlens[i] for i in range(len(txlens))]
reads_treat = [reads_treat[i]*1000000/np.sum(reads_treat) for i in range(len(txlens))]

# Compute rpkms and tpms
rpkms_ctrl = RPKM(reads_ctrl, txlens)
tpms_ctrl = TPM(reads_ctrl, txlens)
rpkms_treat = RPKM(reads_treat, txlens)
tpms_treat = TPM(reads_treat, txlens)

fc_rpkm = FoldChange(rpkms_ctrl, rpkms_treat)
fc_tpm = FoldChange(tpms_ctrl, tpms_treat)

print("\nFold change deteted for each gene")
for i in range(len(txlens)):
    print("Gene %s: FoldChange[RPKM]=%.2f FoldChange[TPM]=%.2f"%(txnames[i], fc_rpkm[i], fc_tpm[i]))          


Fold change deteted for each gene
Gene A: FoldChange[RPKM]=0.91 FoldChange[TPM]=0.95
Gene B: FoldChange[RPKM]=0.91 FoldChange[TPM]=0.95
Gene C: FoldChange[RPKM]=0.91 FoldChange[TPM]=0.95
Gene D: FoldChange[RPKM]=0.91 FoldChange[TPM]=0.95
Gene E: FoldChange[RPKM]=91.12 FoldChange[TPM]=94.65


Now we can start to see an inherent limitation of the normalization we have to do when we perform RNA-seq. While Gene E increased 100-fold, the difference we measure is smaller than that (which metric is closer to the real change, TPM or FPKM?). You'll notice another change as well: it looks like the expression of all the other genes is actually going down! (fold change < 1). Since RNA-seq essentially measures expression of each gene as a fraction of the total pool, huge fluctuations in expression can be difficult or impossible to interpret. If some genes increase expression, it will look like expression of the other genes is decreasing, even if they are actually staying the same. Let's look at one additional even more extreme example: say expression of Gene A doubles from 1000 to 2000:

In [12]:
# Gene info (names, lengths, and transcript counts)
txnames = ["A","B","C","D","E"]
txlens = [5000, 3000, 2000, 10000, 7000]
txcounts_ctrl = [1000, 200, 500, 50, 1]
txcounts_treat = [2000, 200, 500, 50, 1]

# Number of reads observed for each gene
reads_ctrl = [txcounts_ctrl[i]*txlens[i] for i in range(len(txlens))]
reads_ctrl = [reads_ctrl[i]*1000000/np.sum(reads_ctrl) for i in range(len(txlens))]
reads_treat = [txcounts_treat[i]*txlens[i] for i in range(len(txlens))]
reads_treat = [reads_treat[i]*1000000/np.sum(reads_treat) for i in range(len(txlens))]

# Compute rpkms and tpms
rpkms_ctrl = RPKM(reads_ctrl, txlens)
tpms_ctrl = TPM(reads_ctrl, txlens)
rpkms_treat = RPKM(reads_treat, txlens)
tpms_treat = TPM(reads_treat, txlens)

fc_rpkm = FoldChange(rpkms_ctrl, rpkms_treat)
fc_tpm = FoldChange(tpms_ctrl, tpms_treat)

print("\nFold change deteted for each gene")
for i in range(len(txlens)):
    print("Gene %s: FoldChange[RPKM]=%.2f FoldChange[TPM]=%.2f"%(txnames[i], fc_rpkm[i], fc_tpm[i]))          


Fold change deteted for each gene
Gene A: FoldChange[RPKM]=1.17 FoldChange[TPM]=1.27
Gene B: FoldChange[RPKM]=0.59 FoldChange[TPM]=0.64
Gene C: FoldChange[RPKM]=0.59 FoldChange[TPM]=0.64
Gene D: FoldChange[RPKM]=0.59 FoldChange[TPM]=0.64
Gene E: FoldChange[RPKM]=0.59 FoldChange[TPM]=0.64


In reality, genes B-E stayed the same (should have fold-change of 1) and expression of gene A doubled (should have fold-change of 2). However what we see is far from that!

## 3. More on what TPM/FPKM actually compute (optional reading)

The section below is optional reading to understand subtle differences between what is calculated by RPKM vs. TPM.

Consider a cell with $N$ total genes, with $n_i$ copies (transcripts) of each gene $i$ expressed.

Then the total number of transcripts in the cell is $M=\sum_i n_i$, and the percentage of transcripts originating from gene $i$ is $p_i = n_i/M$ and by definition $\sum_i p_i = 1$.

Let $l_i$ be the length of gene $i$. Then we would expect the total number of reads mapping to each gene $r_i$ to be:

$$ r_i = \frac{n_il_i}{\sum_j n_j l_j} * R $$

where $R$ is the number of reads sequenced.

Then we can work out what the RPKM of gene $i$ would be:
$$ rpkm_i = \frac{r_i}{l_i/10^3*R/10^6}  = \frac{n_il_i*R*10^9}{l_i*R\sum_j n_j l_j} = \frac{n_i*10^9}{\sum_j n_j l_j}$$

We can rewrite this in terms of $p_i$ (since $p_i=n_i/M$ and the $M$ cancels):
$$ rpkm_i = \frac{p_i*10^9}{\sum_j p_j l_j} $$

When analyzing RNA-seq, what we really want is something proportional to $p_i$, so we can analyze what fraction of transcripts in our sample come from each gene. So RPKM doesn't quite give us what we want, since there is this weird normalization term ($\sum_j p_j l_j$).

Let's instead consider TPM:

$$ tpm_i = \frac{\frac{n_il_i}{\sum_j n_j l_j} * R}{(l_i/1000)(\sum_j r_j/(l_j/1000)/1000000}$$

Simplifying:
$$tpm_i = \frac{n_i R * 10^6}{\sum_j n_j l_j \sum_j r_j/l_j} $$

We can further simplify the denominator (by substituting relationships defined above):
$\sum_j n_j l_j \sum_j r_j/l_j = R \sum_j n_j l_j \sum_j \frac{n_jl_j}{l_j\sum_j n_j l_j} = R\sum_j n_j$ to get:

$$ tpm_i = \frac{n_i * 10^6}{\sum_j n_j} $$

replacing the $n$'s with $p$'s using the same logic as above and noting $\sum_j p_j=1$, we get:

$$ tpm_i = p_i * 10^6 $$

So, TPM actually gives us an intuitive metric of the percentage of transcripts that come from gene $i$! (just scaled by 1 million to make the numbers easier to deal with).

# Part 2 (Quiz review, 10 pts)

Today's lab is a review for quiz 2. These are the types of questions you can expect.

So that you can know if your answers are correct, in some cases the actual answers are revealed in the autograder tests. But please make sure you understand how the answer was computed so you know how to do it for the quiz!

**Question 1 (1 pt)**: You genotype a SNP in 5,000 people. 200 are homozygous for allele “A” , 1,000 are heterozygous for “A/T”, and the rest are homozygous for “T”. What is the minor allele frequency? Set the value of `maf` to your answer below.

In [19]:
maf = 0
# YOUR CODE HERE
hom_A = 200
A_alleles = (hom_A * 2) + 1000
maf = A_alleles/10000

In [20]:
"""Check value of maf"""
assert(maf>=0 and maf <=1)
assert(maf == (200*2+1000*1)*1.0/10000)

**Question 2 (1 pt)**: For the same SNP as above, what is the genotype frequency of "TT"? Set `freq_tt` to your answer below.

In [21]:
freq_tt = (5000 - 1200) / 5000


In [22]:
"""Check value of freq_tt"""
assert(freq_tt>=0 and freq_tt<=1)
assert(freq_tt == 3800.0/5000)

**Question 3 (1 pt)**: You are performing a GWAS for height, but accidentally shuffled the phenotype (height) values such that each sample is randomly assigned a height value. You test each SNP for assocation with the shuffled phenotypes and obtain a p-value for each SNP. What do you expect the average value of all p-values to be? Set `expected_p` to your answer below.
Hint: what distribution do p-values follow under the null hypothesis of no associations? What is the average of that distribution?

In [23]:
expected_p = 0.5 # set average p-value expected if phenotypes were randomly shuffled
# YOUR CODE HERE


In [24]:
"""Check value of expected_p"""
import numpy as np
assert(expected_p>=0 and expected_p<1)
nullp_avg = np.mean([np.random.random() for i in range(100000)])
assert(abs(expected_p-nullp_avg)<0.01)

**Question 4 (1 pt)**: You are performing a GWAS and test 1 million SNPs for association with a trait. If all SNPs were independent (which is not true in reality), and there is no genetic component of the trait (that is, in reality $\beta=0$ for all SNPs), how many SNPs do you expect to have association p-values <0.05? Set `num_false_pos` to your answer below.

In [32]:
num_false_pos = 1000000*0.05 # Set below
# YOUR CODE HERE

In [33]:
"""Check value of num_false_pos"""
nullp = [np.random.random() for i in range(1000000)]
fp = len([item for item in nullp if item < 0.05])
assert(abs(fp-num_false_pos)<1000) # check that we're pretty close

**Question 5 (1 pt)**: You perform RNA-sequencing using single-end reads of length 36bp and examine the reads mapping to two genes. You aligned 1,000 reads to gene A, which has a length of 5,000bp. You aligned 3,000 reads to gene B, which has a length of 3,000bp. You sequenced 10 million total reads. What is the expression level (in RPKM) of genes A and B? Set your answer to `geneA_RPKM` and `geneB_RPKM` below. 

(Gene A test is visible, Gene B is not)

In [34]:
geneA_RPKM = 1000/((5000/1000)*(10))
geneB_RPKM = 3000/((3)*10)
# Set geneA_RPKM and geneB_RPKM below
# YOUR CODE HERE


In [35]:
"""Check answers to geneA_RPKM and geneB_RPKM"""
assert(geneA_RPKM == 20)

**Question 6 (1 pt)**: You align a read to a reference genome, resulting in the alignment below. What should the CIGAR score be for Read 1? Read 2 and its CIGAR score are given as an example. Set the variable `cigar` to your answer below. (Only use CIGAR characters "M", "I", or "D" in your answer).

```
Reference: ACGCAGTCGGGTCGATTTTCTCTCTCTACTAGCTACGA---ATCGACTGGGTCC
Read 1:            GGGTCGA----CTCTCTCTACTAGC
Read 2:                               ACTAGCTACGAGACATCGACTGGGTCC
```

In [40]:
cigar = "M7D4M14" # set to your answer for read 1
cigar_example = "11M3I13M" # read 2
# YOUR CODE HERE

In [41]:
"""Check cigar"""
assert(cigar_example=="11M3I13M")
assert(cigar.count("M")==2)
assert(cigar.count("D")==1)
assert("14" in cigar)
assert("4" in cigar)
assert("7" in cigar)

In [43]:
%%file regions.txt
chr1 191203 191300
chr2 2000000 2000005
chr3 10000 10004
chr4 60008 60010
chr5 18 25

Overwriting regions.txt


**Question 7 (1 pt)**: The file `regions.txt` is a space delimited file file with chromosomes of the format "chr1", "chr2", etc. (Run the cell above to create it). Write a command that prints the contents of the file to the screen, but removes "chr" from the chromosome names in column 1.  

In [44]:
cmd = "sed 's/chr//g' regions.txt" # change cmd to your answer
# YOUR CODE HERE


In [45]:
"""Test the output of cmd"""
from subprocess import Popen, PIPE
p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
out, err = p.communicate()
assert(p.returncode == 0)
assert(len(str(out).split("\\n"))==6)
out = str(out)[2:]
for i in range(5):
    assert(str(out).split("\\n")[i][0] == "%s"%(i+1))
    assert(len(str(out).split("\\n")[i].split())==3)

**Question 8 (1 pt)**: Write a one-line UNIX command that outputs the median of column 2 for the file `regions.txt`.

In [51]:
cmd = """cat regions.txt | awk -F " " '{print $2}' | datamash median 1""" # change cmd to your answer
# YOUR CODE HERE

In [52]:
"""Check output of cmd"""
from subprocess import Popen, PIPE
p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
out, err = p.communicate()
assert(p.returncode == 0)
assert(int(out.strip())==60008)

**Question 9 (1 pt)**: Write a one-line UNIX command that outputs all rows of `regions.txt` except the row containing "chr3".

In [53]:
cmd = 'cat regions.txt | grep -v "chr3"' # change cmd to your answer
# YOUR CODE HERE

In [54]:
"""Check output of cmd"""
from subprocess import Popen, PIPE
p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
out, err = p.communicate()
assert(p.returncode == 0)
assert(len(str(out).split("\\n")[:-1])==4)
out = str(out)[2:]
assert(str(out).split("\\n")[0][0:4] == "chr1")
assert(str(out).split("\\n")[1][0:4] == "chr2")
assert(str(out).split("\\n")[2][0:4] == "chr4")
assert(str(out).split("\\n")[3][0:4] == "chr5")
for i in range(4):
    assert(len(str(out).split("\\n")[i].split())==3)
assert("chr3" not in out)

**Question 10 (1 pt)**: Match the each of the tools to their appropriate use case.

Tools:
* BWA
* sickle
* samtools
* bcftools
* plink

Use cases:
* [0] Aligning DNA sequencing data
* [1] Performing a GWAS
* [2] Inspecting a BAM file
* [3] Trimming low quality bases from a read
* [4] Counting the number of samples in a VCF file.

Set your answers in the dictionary `toolmatch` below. Some tests are hidden.

In [55]:
toolmatch = {
    "BWA": 0,
    "sickle": 3,
    "samtools": 2,
    "bcftools": 4,
    "plink": 1
}
toolmatch["samtools"] = 2
toolmatch["BWA"] = 0
# YOUR CODE HERE

In [56]:
"""Check toolmatch. A couple tests are hidden."""
for key in ["BWA","sickle","samtools", "bcftools","plink"]:
    assert(toolmatch[key] != -1)
assert(len(toolmatch.keys())==5)
for i in range(5): assert(i in toolmatch.values())
assert(toolmatch["samtools"]==2)
assert(toolmatch["BWA"]==0)